In [30]:
import pandas as pd
import re

import nltk
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords
from Sastrawi.Stemmer.StemmerFactory import StemmerFactory

In [31]:
nltk.download('punkt')
nltk.download('punkt_tab')
nltk.download('stopwords')

[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\Bugi\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package punkt_tab to
[nltk_data]     C:\Users\Bugi\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt_tab is already up-to-date!
[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\Bugi\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [3]:
dataset = pd.read_csv("brimo_reviews.csv")
dataset.head(1)

,reviewId,userName,userImage,content,score,thumbsUpCount,reviewCreatedVersion,at,replyContent,repliedAt,appVersion
0,47252cdd-062f-4f32-b94a-7e5123aea7ba,Pengguna Google,https://play-lh.googleusercontent.com/EGemoI2N...,Aplikasi sangat bagus,5,0,NaN,2024-10-03 18:10:08,NaN,NaN,NaN


In [4]:
dataset.shape

(12000, 11)

In [5]:
dataset.sort_values(by="at", ascending=False, inplace=True)
dataset['at'].head(1)

0    2024-10-03 18:10:08
Name: at, dtype: object

In [10]:
dataset['at'].tail(1)

11999    2024-09-05 03:04:53
Name: at, dtype: object

In [11]:
dataset.columns

Index(['reviewId', 'userName', 'userImage', 'content', 'score',
       'thumbsUpCount', 'reviewCreatedVersion', 'at', 'replyContent',
       'repliedAt', 'appVersion'],
      dtype='object')

In [16]:
used_dataset = dataset.drop(columns=["reviewId", "userName", "userImage", "thumbsUpCount", "reviewCreatedVersion", "at", "replyContent", "repliedAt", "appVersion"])

In [24]:
used_dataset.duplicated().sum()

np.int64(3888)

In [1]:
used_dataset.drop_duplicates(inplace=True)

In [19]:
def clean_text(text):
    text = text.lower()
    text = text.strip() # remove leading/trailing whitespaces
    text = re.sub(r'[^a-z\s]', '', text) # remove non-alphabet characters
    text = re.sub(r'\s+', ' ', text) # remove extra whitespaces
    text = re.sub(r'[^\x00-\x7F]+', '', text) # remove non-ascii characters
    text = re.sub(r'[0-9]+', '', text) # remove numbers
    return text

In [28]:
def rm_stopwords(text):
    stop_words = set(stopwords.words('indonesian'))
    stop_words.update(set(stopwords.words('english')))
    words = word_tokenize(text)
    words = [word for word in words if word not in stop_words]
    return ' '.join(words)

In [44]:
def stemming(text):
    factory = StemmerFactory()
    stemmer = factory.create_stemmer()
    words = word_tokenize(text)
    words = [stemmer.stem(word) for word in words]
    return ' '.join(words)

In [21]:
def fix_word(text):
    corrected = {"batu":"bantu",
                "bromo":"brimo",
                "n":"dan",
                "g":"tidak",
                "jt":"juta",
                "dongg":"dong",
                "yg":"yang",
                "gw":"saya",
                "anjgg":"anjing",
                "bgt":"banget",
                "gak":"tidak",
                "kalo":"kalau",
                "bgtu":"begitu",
                "dri":"dari",
                "pdahal":"padahal",
                "ngepass":"ngepas",
                "bngt":"banget",
                "tolongg":"tolong",
                "mantulll":"mantul",
                "asyik":"asik",
                "skrg":"sekarang",
                "terusss":"terus",
                "prktis":"praktis",
                "tk":"thank you",
                "mas":"masa"}
    words = word_tokenize(text)
    for index, word in enumerate(words):
        if word in corrected:
            words[index] = corrected[word]
    return ' '.join(words)

In [33]:
used_dataset.content.head(10)

0                                Aplikasi sangat bagus
1    Saya beli token listrik tidak bisa di gunakan ...
2                                         Sangat bagus
3                     Aplikasi nya Keren, tf saldo min
4                                       Sangat membatu
5     Mantappp transaksinya cepat dan gak pernah rewel
6                                      So far soo good
7                                            Bromo oke
8                                      membantu banget
9                                     Sangat2 membantu
Name: content, dtype: object

In [45]:
clean_dataset = used_dataset.copy()
clean_dataset.content = used_dataset.content.apply(clean_text)
clean_dataset.content = clean_dataset.content.apply(rm_stopwords)
# clean_dataset.content = clean_dataset.content.apply(stemming)

In [35]:
clean_dataset.to_csv("clean_brimo_reviews.csv", index=False)

In [46]:
clean_dataset.content.head(10)

0                        aplikasi bagus
1        beli token listrik udah sukses
2                                 bagus
3       aplikasi nya keren tf saldo min
4                                  batu
5    mantappp transaksi cepat gak rewel
6                          far soo good
7                             bromo oke
8                          bantu banget
9                                 bantu
Name: content, dtype: object

In [33]:
cleaned_dataset = pd.read_csv("clean_brimo_reviews.csv")

In [36]:
cleaned_dataset.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 8112 entries, 0 to 8111
Data columns (total 2 columns):
 #   Column   Non-Null Count  Dtype 
---  ------   --------------  ----- 
 0   content  7962 non-null   object
 1   score    8112 non-null   int64 
dtypes: int64(1), object(1)
memory usage: 126.9+ KB


In [37]:
cleaned_dataset.dropna(inplace=True)

In [38]:
cleaned_dataset.info()

<class 'pandas.core.frame.DataFrame'>
Index: 7962 entries, 0 to 8111
Data columns (total 2 columns):
 #   Column   Non-Null Count  Dtype 
---  ------   --------------  ----- 
 0   content  7962 non-null   object
 1   score    7962 non-null   int64 
dtypes: int64(1), object(1)
memory usage: 186.6+ KB


In [39]:
cleaned_dataset.content = cleaned_dataset.content.apply(fix_word)

In [40]:
cleaned_dataset.head(10)

,content,score
0,aplikasi bagus,5
1,beli token listrik udah sukses,3
2,bagus,5
3,aplikasi nya keren tf saldo min,5
4,bantu,5
5,mantappp transaksi cepat tidak rewel,5
6,far soo good,4
7,brimo oke,5
8,bantu banget,5
9,bantu,5


In [41]:
cleaned_dataset.to_csv("clean_brimo_reviews.csv", index=False)